In [1]:
from prompt import *
from LLM import *

table_plan_map = {
    'company_info': 0,
    'company_register': 1,
    'sub_company_info': 2,
    'legal_document': 3,
    'legal_abstract': 4,
    'xzgxf_info': 5,
    'legal_document_list': 6,
    'court_info': 7,
    'court_code': 8,
    'lawfirm_info': 9,
    'lawfirm_log': 10,
    'address_info': 11,
    'address_code': 12,
    'temp_info': 13
    }

In [2]:
rsp = LLM(TABLE_PROMPT.format(question="(2021)辽01民终16020号案件中，审理当天审理法院的天气情况是？本题目调用了几类API？串行了几个ＡＰＩ？串行了几次？"))

In [3]:
fcts = prase_json_from_response(rsp=rsp)
fcts

['legal_document', 'temp_info']

In [4]:
plan_id = [table_plan_map[fct] for fct in fcts]
plan_id

[3, 13]